In [48]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [2]:
donors = pd.read_csv(r'C:\Users\Anderson\Desktop\learningSet.csv\learningSet.csv')
donors

C:\Users\Anderson\AppData\Local\Temp\ipykernel_14124\980717866.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  donors = pd.read_csv(r'C:\Users\Anderson\Desktop\learningSet.csv\learningSet.csv')


,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,GRI,0,IL,61081,,,3712,0,,...,0.0,0,L,4,E,X,X,X,39.0,C
1,9401,BOA,1,CA,91326,,,5202,0,,...,0.0,0,L,2,G,X,X,X,1.0,A
2,9001,AMH,1,NC,27017,,,0,0,,...,0.0,1,L,4,E,X,X,X,60.0,C
3,8701,BRY,0,CA,95953,,,2801,0,,...,0.0,1,L,4,E,X,X,X,41.0,C
4,8601,,0,FL,33176,,,2001,0,X,...,0.0,1,L,2,F,X,X,X,26.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,9601,ASE,1,AK,99504,,,0,0,,...,0.0,0,L,1,G,X,X,X,12.0,C
95408,9601,DCD,1,TX,77379,,,5001,0,,...,0.0,1,L,1,F,X,X,X,2.0,A
95409,9501,MBC,1,MI,48910,,,3801,0,,...,0.0,1,L,3,E,X,X,X,34.0,B
95410,8601,PRV,0,CA,91320,,,4005,0,X,...,18.0,1,L,4,F,X,X,X,11.0,A


In [4]:
donors.dtypes

ODATEDW       int64
OSOURCE      object
TCODE         int64
STATE        object
ZIP          object
             ...   
MDMAUD_R     object
MDMAUD_F     object
MDMAUD_A     object
CLUSTER2    float64
GEOCODE2     object
Length: 481, dtype: object

In [5]:
print(donors.isnull().sum())

ODATEDW       0
OSOURCE       0
TCODE         0
STATE         0
ZIP           0
           ... 
MDMAUD_R      0
MDMAUD_F      0
MDMAUD_A      0
CLUSTER2    132
GEOCODE2    132
Length: 481, dtype: int64


In [7]:
donors = donors.drop_duplicates()

In [8]:
donors.isna().sum().sum()

5158796

In [9]:
null_df = donors.isna().sum()/len(donors)
null_df = pd.DataFrame(null_df).reset_index()
null_df.columns = ['column_names','percentage_of_nulls']
null_df = null_df.sort_values(by = 'percentage_of_nulls', ascending = False )
null_df

,column_names,percentage_of_nulls
414,RDATE_5,0.999906
436,RAMNT_5,0.999906
412,RDATE_3,0.997464
434,RAMNT_3,0.997464
413,RDATE_4,0.997055
...,...,...
168,ETHC3,0.000000
167,ETHC2,0.000000
166,ETHC1,0.000000
165,HHD12,0.000000


In [10]:
threshold = 0.3
condition = null_df['percentage_of_nulls'] > threshold
columns_above_threshold = null_df[condition]
columns_above_threshold

,column_names,percentage_of_nulls
414,RDATE_5,0.999906
436,RAMNT_5,0.999906
412,RDATE_3,0.997464
434,RAMNT_3,0.997464
413,RDATE_4,0.997055
...,...,...
372,ADATE_13,0.421530
383,ADATE_24,0.387509
380,ADATE_21,0.369052
364,ADATE_5,0.352052


In [11]:
drop_column_list = list(columns_above_threshold['column_names'])
donors = donors.drop(columns = drop_column_list)

In [12]:
donors.replace(' ', np.nan, inplace=True)
null_df = donors.isna().sum()/len(donors)
null_df = pd.DataFrame(null_df).reset_index()
null_df.columns = ['column_names','percentage_of_nulls']
null_df = null_df.sort_values(by = 'percentage_of_nulls', ascending = False )
threshold = 0.3
condition = null_df['percentage_of_nulls'] > threshold
columns_above_threshold = null_df[condition]
columns_above_threshold

,column_names,percentage_of_nulls
11,RECPGVG,0.998805
34,SOLP3,0.998113
36,MAJOR,0.996919
55,PLATES,0.994131
42,HOMEE,0.990703
54,CARDS,0.989089
19,CHILD03,0.987989
5,MAILCODE,0.985337
6,PVASTATE,0.984719
53,KIDSTUFF,0.983901


In [13]:
drop_column_list = list(columns_above_threshold['column_names'])
donors = donors.drop(columns = drop_column_list)
donors

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,DOB,NOEXCH,MDMAUD,DOMAIN,CLUSTER,...,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,GRI,0,IL,61081,3712,0,XXXX,T2,36,...,0.0,0,L,4,E,X,X,X,39.0,C
1,9401,BOA,1,CA,91326,5202,0,XXXX,S1,14,...,0.0,0,L,2,G,X,X,X,1.0,A
2,9001,AMH,1,NC,27017,0,0,XXXX,R2,43,...,0.0,1,L,4,E,X,X,X,60.0,C
3,8701,BRY,0,CA,95953,2801,0,XXXX,R2,44,...,0.0,1,L,4,E,X,X,X,41.0,C
4,8601,NaN,0,FL,33176,2001,0,XXXX,S2,16,...,0.0,1,L,2,F,X,X,X,26.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,9601,ASE,1,AK,99504,0,0,XXXX,C2,27,...,0.0,0,L,1,G,X,X,X,12.0,C
95408,9601,DCD,1,TX,77379,5001,0,XXXX,C1,24,...,0.0,1,L,1,F,X,X,X,2.0,A
95409,9501,MBC,1,MI,48910,3801,0,XXXX,C3,30,...,0.0,1,L,3,E,X,X,X,34.0,B
95410,8601,PRV,0,CA,91320,4005,0,XXXX,C1,24,...,18.0,1,L,4,F,X,X,X,11.0,A


In [14]:
categorical_df = donors.select_dtypes(exclude=['number'])
categorical_df

,OSOURCE,STATE,ZIP,NOEXCH,MDMAUD,DOMAIN,CLUSTER,HOMEOWNR,GENDER,DATASRCE,...,RFA_17,RFA_18,RFA_19,RFA_22,RFA_2R,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,GRI,IL,61081,0,XXXX,T2,36,NaN,F,NaN,...,S4E,S4E,S4E,S4E,L,E,X,X,X,C
1,BOA,CA,91326,0,XXXX,S1,14,H,M,3,...,NaN,NaN,N1E,N1E,L,G,X,X,X,A
2,AMH,NC,27017,0,XXXX,R2,43,U,M,3,...,NaN,S4D,S4D,S4D,L,E,X,X,X,C
3,BRY,CA,95953,0,XXXX,R2,44,U,F,3,...,S2D,S2D,A1D,A1D,L,E,X,X,X,C
4,NaN,FL,33176,0,XXXX,S2,16,H,F,3,...,A2D,A2D,A3D,I4E,L,F,X,X,X,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,ASE,AK,99504,0,XXXX,C2,27,NaN,M,NaN,...,NaN,NaN,NaN,NaN,L,G,X,X,X,C
95408,DCD,TX,77379,0,XXXX,C1,24,H,M,3,...,NaN,NaN,NaN,NaN,L,F,X,X,X,A
95409,MBC,MI,48910,0,XXXX,C3,30,NaN,M,NaN,...,NaN,F1D,NaN,P1D,L,E,X,X,X,B
95410,PRV,CA,91320,0,XXXX,C1,24,H,F,2,...,S2F,S2F,A1F,A1F,L,F,X,X,X,A


In [15]:
categorical_columns = donors.select_dtypes(exclude=['number']).columns.tolist()
for column in categorical_columns:
    unique_values = donors[column].unique()
    print(f"Unique values for {column}:")
    print(unique_values)
    print()

Unique values for OSOURCE:
['GRI' 'BOA' 'AMH' 'BRY' nan 'CWR' 'DRK' 'NWN' 'LIS' 'MSD' 'AGR' 'CSM'
 'ENQ' 'HCC' 'USB' 'FRC' 'RKB' 'PCH' 'AMB' 'L15' 'BBK' 'L21' 'SYN' 'L01'
 'MOP' 'UCA' 'ESN' 'IMP' 'AVN' 'RMG' 'DNA' 'L04' 'AML' 'AIR' 'DUR' 'LHJ'
 'WKB' 'STL' 'DCD' 'IMA' 'ACS' 'ALZ' 'NEX' 'HAR' 'SGI' 'MBC' 'BSH' 'NAD'
 'HOS' 'HHL' 'GRT' 'L02' 'APP' 'DAC' 'BHG' 'NSH' 'NPT' 'L16' 'PV3' 'LOG'
 'ASC' 'AGS' 'ARG' 'DON' 'VIK' 'ARB' 'HHH' 'ANT' 'WRG' 'PBL' 'OMH' 'CRG'
 'UBA' 'ASH' 'COK' 'RPH' 'STV' 'NAS' 'SSS' 'LEO' 'KNG' 'KIP' 'ASS' 'GDA'
 'STR' 'CAW' 'GET' 'HAN' 'DEL' 'FLD' 'L25' 'MER' 'SYC' 'HAM' 'PSM' 'HIL'
 'SPN' 'DNB' 'GPS' 'ASP' 'INN' 'ABW' 'CFI' 'JFY' 'LAK' 'LVT' 'RED' 'TIM'
 'MON' 'MM3' 'FAR' 'MTR' 'HOW' 'FOR' 'LKE' 'DBL' 'K3M' 'PGR' 'ADD' 'IML'
 'SMZ' 'CNT' 'SUN' 'MCC' 'BEL' 'TVF' 'TRN' 'PCL' 'HRB' 'OVS' 'WFD' 'TX2'
 'NWF' 'KEN' 'NSN' 'NEW' 'CJR' 'NHB' 'FCR' 'BSM' 'SIG' 'CHT' 'CAP' 'TVG'
 'SUF' 'PRV' 'TRO' 'GUR' 'WIG' 'MAT' 'D02' 'GLP' 'HEA' 'BLI' 'EAS' 'SFH'
 'PBK' 'TOR' 'HJR' 'MCO' '

In [17]:
def check_column_type(series):
    """
    Returns:
        str: "numeric" if all non-null values are numeric,
             "letters" if all non-null values are letters,
             "mixed" if contains both letters and numbers, or 
             "other" if it's neither numeric nor letters.
    """
    non_null_values = series.dropna().astype(str)  # Drop NaN values and convert to string

    is_numeric = non_null_values.str.isdigit().all()  # Check if all are digits
    is_alpha = non_null_values.str.isalpha().all()    # Check if all are letters

    if is_numeric:
        return "numeric"
    elif is_alpha:
        return "letters"
    elif any(non_null_values.str.isdigit()) and any(non_null_values.str.isalpha()):
        return "mixed"
    else:
        return "other"

# Apply the function to each column and create a new DataFrame
column_types = pd.DataFrame({'Column': categorical_df.columns,
                             'Type': [check_column_type(categorical_df[col]) for col in categorical_df.columns]})

print(column_types.to_markdown(index=False))


| Column   | Type    |
|:---------|:--------|
| OSOURCE  | other   |
| STATE    | letters |
| ZIP      | other   |
| NOEXCH   | mixed   |
| MDMAUD   | other   |
| DOMAIN   | other   |
| CLUSTER  | numeric |
| HOMEOWNR | letters |
| GENDER   | letters |
| DATASRCE | numeric |
| RFA_2    | other   |
| RFA_3    | other   |
| RFA_4    | other   |
| RFA_6    | other   |
| RFA_7    | other   |
| RFA_8    | other   |
| RFA_9    | other   |
| RFA_11   | other   |
| RFA_12   | other   |
| RFA_14   | other   |
| RFA_16   | other   |
| RFA_17   | other   |
| RFA_18   | other   |
| RFA_19   | other   |
| RFA_22   | other   |
| RFA_2R   | letters |
| RFA_2A   | letters |
| MDMAUD_R | letters |
| MDMAUD_F | mixed   |
| MDMAUD_A | letters |
| GEOCODE2 | letters |


In [18]:
drop_list = []
drop_list.append('OSOURCE')
drop_list.append('ZIP')
categorical_df.drop(columns=drop_list, inplace=True)

In [19]:
print (categorical_df['GENDER'].value_counts())
print (categorical_df['GENDER'].isna().sum())

GENDER
F    51277
M    39094
U     1715
J      365
C        2
A        2
Name: count, dtype: int64
2957


In [20]:
categorical_df['GENDER'] = categorical_df['GENDER'].apply(lambda x: x if x in ['M', 'F'] else 'other')

In [21]:
print (categorical_df['GENDER'].value_counts())
print (categorical_df['GENDER'].isna().sum())

GENDER
F        51277
M        39094
other     5041
Name: count, dtype: int64
0


In [22]:
print (categorical_df['GEOCODE2'].value_counts())
print (categorical_df['GEOCODE2'].dtype)
print (categorical_df['GEOCODE2'].isna().sum())

GEOCODE2
A    34484
B    28505
D    16580
C    15524
Name: count, dtype: int64
object
319


In [23]:
categorical_df = categorical_df[categorical_df['GEOCODE2'].notna()]
categorical_df['GEOCODE2'].replace('nan', 'A', inplace = True)

C:\Users\Anderson\AppData\Local\Temp\ipykernel_14124\2572918343.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categorical_df['GEOCODE2'].replace('nan', 'A', inplace = True)


In [24]:
print (categorical_df['GEOCODE2'].value_counts())
print (categorical_df['GEOCODE2'].isna().sum())

GEOCODE2
A    34484
B    28505
D    16580
C    15524
Name: count, dtype: int64
0


In [25]:
categorical_df.columns

Index(['STATE', 'NOEXCH', 'MDMAUD', 'DOMAIN', 'CLUSTER', 'HOMEOWNR', 'GENDER',
       'DATASRCE', 'RFA_2', 'RFA_3', 'RFA_4', 'RFA_6', 'RFA_7', 'RFA_8',
       'RFA_9', 'RFA_11', 'RFA_12', 'RFA_14', 'RFA_16', 'RFA_17', 'RFA_18',
       'RFA_19', 'RFA_22', 'RFA_2R', 'RFA_2A', 'MDMAUD_R', 'MDMAUD_F',
       'MDMAUD_A', 'GEOCODE2'],
      dtype='object')

In [26]:
categorical_df.isna().sum().sum()

236154

In [27]:
null_cat = categorical_df.isna().sum()/len(donors)
null_cat = pd.DataFrame(null_cat).reset_index()
null_cat.columns = ['column_names','percentage_of_nulls']
null_cat = null_cat.sort_values(by = 'percentage_of_nulls', ascending = False )
threshold = 0.3
condition = null_cat['percentage_of_nulls'] < threshold
columns_below_threshold_cat = null_cat[condition]
columns_below_threshold_cat

,column_names,percentage_of_nulls
19,RFA_17,0.288716
22,RFA_22,0.267954
21,RFA_19,0.255649
5,HOMEOWNR,0.231218
20,RFA_18,0.222016
7,DATASRCE,0.221324
18,RFA_16,0.213223
17,RFA_14,0.196946
14,RFA_9,0.117375
15,RFA_11,0.108760


In [28]:
from sklearn.impute import SimpleImputer

def impute_mode_for_categorical(df, threshold=0.1):
 
    for column in df.columns:
        missing_percentage = df[column].isnull().mean()
        
        if missing_percentage > threshold:
            continue  # Skip columns with missing values above the threshold
        
        imputer = SimpleImputer(strategy='most_frequent')
        df[column] = imputer.fit_transform(df[[column]]).ravel()


impute_mode_for_categorical(categorical_df, threshold=0.1)

categorical_df.isna().sum().sum()

C:\Users\Anderson\AppData\Local\Temp\ipykernel_14124\1163331409.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = imputer.fit_transform(df[[column]]).ravel()
C:\Users\Anderson\AppData\Local\Temp\ipykernel_14124\1163331409.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = imputer.fit_transform(df[[column]]).ravel()
C:\Users\Anderson\AppData\Local\Temp\ipykernel_14124\1163331409.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

202577

In [29]:
def proportional_imputation(df):
    for column in df.columns:
        # Calculate the proportions of each category in the non-NaN values
        value_counts = df[column].value_counts(normalize=True)
        
        # Get the indices of NaN values
        nan_indices = df[df[column].isna()].index
        
        # Sample categories based on their proportions to fill NaN values
        fill_values = np.random.choice(value_counts.index, size=len(nan_indices), p=value_counts.values)
        
        # Fill the NaN values with the sampled categories
        df.loc[nan_indices, column] = fill_values
        
    return df

categorical_df = proportional_imputation(categorical_df)

In [30]:
categorical_df.isna().sum().sum()

0

In [31]:
categorical_df.shape

(95093, 29)

In [32]:
categorical_df = categorical_df.astype(str)

In [33]:
numerical_df = donors.select_dtypes(include=['number'])
numerical_df

,ODATEDW,TCODE,DOB,AGE,INCOME,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,...,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,TARGET_B,TARGET_D,HPHONE_D,RFA_2F,CLUSTER2
0,8901,0,3712,60.0,NaN,0,0,39,34,18,...,8911,9003.0,4.0,7.741935,95515,0,0.0,0,4,39.0
1,9401,1,5202,46.0,6.0,16,0,15,55,11,...,9310,9504.0,18.0,15.666667,148535,0,0.0,0,2,1.0
2,9001,1,0,NaN,3.0,2,0,20,29,33,...,9001,9101.0,12.0,7.481481,15078,0,0.0,1,4,60.0
3,8701,0,2801,70.0,1.0,2,0,23,14,31,...,8702,8711.0,9.0,6.812500,172556,0,0.0,1,4,41.0
4,8601,0,2001,78.0,3.0,60,1,28,9,53,...,7903,8005.0,14.0,6.864865,7112,0,0.0,1,2,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,9601,1,0,NaN,NaN,0,14,36,47,11,...,9602,NaN,NaN,25.000000,184568,0,0.0,0,1,12.0
95408,9601,1,5001,48.0,7.0,1,0,31,43,19,...,9603,NaN,NaN,20.000000,122706,0,0.0,1,1,2.0
95409,9501,1,3801,60.0,NaN,0,0,18,46,20,...,9410,9501.0,3.0,8.285714,189641,0,0.0,1,3,34.0
95410,8601,0,4005,58.0,7.0,0,0,28,35,20,...,8612,8704.0,4.0,12.146341,4693,1,18.0,1,4,11.0


In [34]:
numerical_df.nunique()

ODATEDW      54
TCODE        55
DOB         947
AGE          96
INCOME        7
           ... 
TARGET_B      2
TARGET_D     71
HPHONE_D      2
RFA_2F        4
CLUSTER2     62
Length: 338, dtype: int64

In [35]:
numerical_df['TARGET_B'].isna().sum()

0

In [37]:
numerical_df.groupby('TARGET_B')['TARGET_D'].count()

TARGET_B
0    90569
1     4843
Name: TARGET_D, dtype: int64

In [38]:
target_1 = numerical_df[numerical_df['TARGET_B'] == 1]
target_1['TARGET_D'].describe()

count    4843.000000
mean       15.624344
std        12.445137
min         1.000000
25%        10.000000
50%        13.000000
75%        20.000000
max       200.000000
Name: TARGET_D, dtype: float64

In [39]:
target_0 = numerical_df[numerical_df['TARGET_B'] == 0]
target_0['TARGET_D'].describe()

count    90569.0
mean         0.0
std          0.0
min          0.0
25%          0.0
50%          0.0
75%          0.0
max          0.0
Name: TARGET_D, dtype: float64

In [40]:
target = pd.DataFrame(numerical_df['TARGET_B'])
target

,TARGET_B
0,0
1,0
2,0
3,0
4,0
...,...
95407,0
95408,0
95409,0
95410,1


In [41]:
numerical_df = numerical_df.drop (['TARGET_B','TARGET_D'], axis = 1)

In [42]:
numerical_df.isna().sum().sum()

254370

In [43]:
null_num = numerical_df.isna().sum()/len(donors)
null_num = pd.DataFrame(null_num).reset_index()
null_num.columns = ['column_names','percentage_of_nulls']
null_num = null_num.sort_values(by = 'percentage_of_nulls', ascending = False )
threshold = 0.3
condition = null_num['percentage_of_nulls'] < threshold
columns_below_threshold_num = null_num[condition]
columns_below_threshold_num

,column_names,percentage_of_nulls
310,ADATE_17,0.289796
313,ADATE_22,0.268813
312,ADATE_19,0.256572
3,AGE,0.248030
4,INCOME,0.223096
...,...,...
111,HVP2,0.000000
110,HVP1,0.000000
109,ETHC6,0.000000
108,ETHC5,0.000000


In [44]:
def impute_mean_for_numerical(df, threshold=0.1):
 
    for column in df.columns:
        missing_percentage = df[column].isnull().mean()
        
        if missing_percentage > threshold:
            continue  # Skip columns with missing values above the threshold
        
        # Check if the column is numeric using np.issubdtype
        if np.issubdtype(df[column].dtype, np.number):
            imputer = SimpleImputer(strategy='mean')
            df[column] = imputer.fit_transform(df[[column]]).ravel()


impute_mean_for_numerical(numerical_df, threshold=0.1)

numerical_df.isna().sum().sum()

224836

In [45]:
numerical_df = proportional_imputation(numerical_df)      

In [46]:
numerical_df.isna().sum().sum()

0

In [47]:
numerical_df.shape

(95412, 336)

In [49]:
scaler = StandardScaler()
normalized_num = scaler.fit_transform(numerical_df)
normalized_num = pd.DataFrame(normalized_num, columns=numerical_df.columns)

In [50]:
donors_clean = pd.concat([target, normalized_num, categorical_df], axis=1) 
donors_clean

,TARGET_B,ODATEDW,TCODE,DOB,AGE,INCOME,HIT,MALEMILI,MALEVET,VIETVETS,...,RFA_17,RFA_18,RFA_19,RFA_22,RFA_2R,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,0,-0.699843,-0.056847,0.463551,-0.098461,0.600310,-0.356881,-0.206977,0.745798,0.284659,...,S4E,S4E,S4E,S4E,L,E,X,X,X,C
1,0,0.755960,-0.055799,1.162350,-0.937004,1.139076,1.362283,-0.206977,-1.346527,1.675602,...,A1F,A1C,N1E,N1E,L,G,X,X,X,A
2,0,-0.408683,-0.055799,-1.277349,-0.098461,-0.477222,-0.141985,-0.206977,-0.910626,-0.046518,...,S4B,S4D,S4D,S4D,L,E,X,X,X,C
3,0,-1.282165,-0.056847,0.036299,0.500499,-1.554755,-0.141985,-0.206977,-0.649086,-1.040049,...,S2D,S2D,A1D,A1D,L,E,X,X,X,C
4,0,-1.573325,-0.056847,-0.338895,0.979666,-0.477222,6.089983,-0.009563,-0.213185,-1.371225,...,A2D,A2D,A3D,I4E,L,F,X,X,X,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,0,1.338282,-0.055799,-1.277349,-1.176588,-1.554755,-0.356881,2.556819,0.484257,1.145719,...,L3F,S3E,S2E,A2F,L,G,X,X,X,C
95408,0,1.338282,-0.055799,1.068082,-0.817212,1.677842,-0.249433,-0.206977,0.048356,0.880777,...,S4F,A1E,A2F,S4E,L,F,X,X,X,A
95409,0,1.047121,-0.055799,0.505291,-0.098461,0.600310,-0.356881,-0.206977,-1.084987,1.079483,...,P1D,F1D,A1D,P1D,L,E,X,X,X,B
95410,1,-1.573325,-0.056847,0.600966,-0.218252,1.677842,-0.356881,-0.206977,-0.213185,0.350894,...,S2F,S2F,A1F,A1F,L,F,X,X,X,A


In [51]:
y = donors_clean['TARGET_B']
X = donors_clean.drop(['TARGET_B'], axis = 1)

# Get dummies
numericalX = X.select_dtypes(include = [np.number])
categoricalX = X.select_dtypes(exclude = [np.number])

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first').fit(categoricalX)
encoded_categorical = encoder.transform(categoricalX).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical)

# Convert only integer column names to strings
encoded_categorical.columns = [str(col) if isinstance(col, int) else col for col in encoded_categorical.columns]
X = pd.concat([numericalX, encoded_categorical], axis = 1)
X.isna().sum().sum() 

0

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

train = pd.concat([X_train, y_train],axis=1)
print(train['TARGET_B'].value_counts())
train

TARGET_B
0    72486
1     3843
Name: count, dtype: int64


,ODATEDW,TCODE,DOB,AGE,INCOME,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,...,1631,1632,1633,1634,1635,1636,1637,1638,1639,TARGET_B
79401,-1.573325,-0.055799,-0.198198,0.799979,0.061544,-0.141985,-0.206977,0.048356,-1.304990,1.996857,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0
86429,1.338282,-0.056847,0.833586,-0.517732,0.600310,-0.356881,-0.206977,-0.126004,-0.377695,0.302800,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0
76729,-0.991004,-0.055799,0.462144,-0.098461,0.600310,0.180358,-0.206977,1.007338,-0.907578,1.658045,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1
38838,-0.699843,-0.055799,-0.291996,0.919771,1.139076,0.610149,-0.009563,0.832978,-0.443930,0.472206,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
83012,0.755960,-0.053702,0.739787,-0.397940,0.600310,1.899522,6.307686,-0.300365,-0.046518,-0.826571,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21243,-1.573325,-0.056847,-0.007318,0.500499,-0.477222,0.180358,0.187851,0.571437,-1.304990,-0.036011,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0
45891,1.338282,-0.054750,0.223896,0.260915,-0.477222,-0.356881,-0.206977,0.484257,-1.238755,1.658045,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0
42613,-1.282165,-0.055799,-1.277349,1.638522,0.600310,-0.356881,-0.206977,0.397077,-0.046518,0.528674,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1
43567,-1.573325,-0.055799,-0.289182,0.859875,0.600310,-0.249433,-0.206977,2.489402,-0.510166,-0.092480,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0


In [53]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train, y_train)
pred = LR.predict(X_test)
LR.score(X_test, y_test)

C:\Users\Anderson\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9475973379447676

In [54]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

print("precision: ",precision_score(y_test,pred))
print("recall: ",recall_score(y_test,pred))
print("f1: ",f1_score(y_test,pred))

precision:  0.5
recall:  0.001
f1:  0.001996007984031936


In [55]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state =0,sampling_strategy=1) 

In [56]:
X_train_SMOTE,y_train_SMOTE = sm.fit_resample(X_train,y_train)

In [57]:
train_smote = pd.concat([X_train_SMOTE,y_train_SMOTE], axis = 1)

In [58]:
LR_smote = LogisticRegression()
LR_smote.fit(X_train_SMOTE, y_train_SMOTE)
print (LR_smote.score(X_test, y_test))

C:\Users\Anderson\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6395220877220563


In [59]:
pred_smote = LR_smote.predict(X_test)

print("precision: ",precision_score(y_test,pred_smote))
print("recall: ",recall_score(y_test,pred_smote))
print("f1: ",f1_score(y_test,pred_smote))

precision:  0.06714769547931085
recall:  0.456
f1:  0.11705814401232191


In [60]:
category_0 = train[train['TARGET_B'] == 0]
category_1 = train[train['TARGET_B'] == 1]
from sklearn.utils import resample
category_0_under = resample (category_0, replace = False, n_samples = len(category_1), random_state = 0)
train_undersample = pd.concat([category_0_under, category_1], axis = 0)

In [61]:
y_train_undersample = train_undersample['TARGET_B']
X_train_undersample = train_undersample.drop(['TARGET_B'], axis = 1)
LR_under = LogisticRegression()
LR_under.fit(X_train_undersample, y_train_undersample)
print (LR_under.score(X_test, y_test))

C:\Users\Anderson\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5504375622281612


In [62]:
pred_under = LR_under.predict(X_test)
print("precision: ",precision_score(y_test,pred_under))
print("recall: ",recall_score(y_test,pred_under))
print("f1: ",f1_score(y_test,pred_under))

precision:  0.06437521554201632
recall:  0.56
f1:  0.11547582224971646
